# Setup

In [1]:
import sys
assert sys.version_info >=(3, 7)

import numpy as np
import cv2 as cv
from util_func import *
import imutils 

In [ ]:
pip install imutils

## Exercise 1 
Rotate image by 45 degrees without cropping the sides of the image. (Hint: There are 2 strategies to tackle these problems). Use "lena.jfif" as the input image.
- Use external libraries imutils.
- Modify the transformation matrix.

In [2]:
# imutils
img = cv.imread("images/lena.jfif")

rotated = imutils.rotate_bound(img, 45)
cv.imshow("Imutils Rotation", rotated)
cv.waitKey(0)


-1

In [5]:
# Load the image
img = cv.imread("images/lena.jfif")

# Determine the center of the image
(h, w) = img.shape[:2]
center = (w // 2, h // 2)

# Define the rotation angle in degrees (positive for counter-clockwise)
angle = 45

# Get the rotation matrix
M = cv.getRotationMatrix2D(center, angle, 1.0)

# Calculate the new image dimensions after rotation
cos_theta = abs(M[0, 0])
sin_theta = abs(M[0, 1])
new_w = int((h * sin_theta) + (w * cos_theta))
new_h = int((h * cos_theta) + (w * sin_theta))

# Adjust the rotation matrix to account for translation and padding
M[0, 2] += (new_w - w) // 2
M[1, 2] += (new_h - h) // 2

# Perform the rotation with padding to avoid cropping
r_img = cv.warpAffine(img, M, (new_w, new_h))

# Display the original and rotated images
cv.imshow("Rotated Image", r_img)
cv.waitKey(0)
cv.destroyAllWindows()

## Exercise 2
Use the images with titles: "flower.jfif" and "native-bee.png". I want to put flower above an image. If I add two images, it will change color. If I blend it, I get a transparent effect. But I want it to be opaque. If it was a rectangular region, we could use the ROI as we did in the previous section. But flower is not a rectangular region. This is where bitwise operations, like AND, OR, NOT and XOR really come in handy. The associated functions are cv.bitwise_and(), cv.bitwise_or() and cv.bitwise_not(). You need to use cv.threshold function to segment the flower. Please refer to online documentation for more info. The result should resemble the following:

In [3]:
# Load two images
img1 = cv.imread('images/native-bee.png')
img2 = cv.imread('images/flower.jfif')

assert img1 is not None, "file could not be read, check with os.path.exists()"
assert img2 is not None, "file could not be read, check with os.path.exists()"

# I want to put logo on top-left corner, So I create a ROI
rows,cols,channels = img2.shape
roi = img1[0:rows, 0:cols]

# Now create a mask of logo and create its inverse mask also
img2gray = cv.cvtColor(img2,cv.COLOR_BGR2GRAY)
ret, mask = cv.threshold(img2gray, 10, 255, cv.THRESH_BINARY)
mask_inv = cv.bitwise_not(mask)

# Now black-out the area of logo in ROI
img1_bg = cv.bitwise_and(roi,roi,mask = mask_inv)

# Take only region of logo from logo image.
img2_fg = cv.bitwise_and(img2,img2,mask = mask)

# Put logo in ROI and modify the main image
dst = cv.add(img1_bg,img2_fg)
img1[0:rows, 0:cols ] = dst
cv.imshow('res',img1)
cv.waitKey(0)
cv.destroyAllWindows()

In [4]:
kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
kernel2 = np.array([[-1, -1, -1, -1, -1], [-1, -1, -1, -1, -1], [-1, -1, 25, -1, -1],[-1, -1, -1, -1, -1], [-1, -1, -1, -1, -1]])

sharpen = cv.filter2D(img1, -1, kernel)
sharpen2 = cv.filter2D(img1, -1, kernel2)


cv.imshow("sharpen", sharpen)
cv.imshow("sharpen2", sharpen2)
cv.waitKey(0)
cv.destroyAllWindows()

5x5 has a wider aperture which makes it enhances the edges and details more aggresive than 3x3. It enhances larger features and edges in the image compared to 3x3 but it also introduced more noises compared to 3x3 

## Exercise 4

Apply different image smoothing techniques (e.g. average filter, Gaussian kernel and median filter) on 'noise_lena.jpg' and display the resulting images after the convolution. Comment on the outcomes and deduce the type of noise present on the image.


In [5]:
# Load the image
image_path = "images/noise_lena.jpg"
image = cv.imread(image_path)

# Apply average filter (blur)
average_filtered = cv.blur(image, (5, 5))

# Apply Gaussian filter
gaussian_filtered = cv.GaussianBlur(image, (5, 5), 0)

# Apply median filter
median_filtered = cv.medianBlur(image, 5)

# Display the original and filtered images
cv.imshow("Original Image", image)
cv.imshow("Average Filtered", average_filtered)
cv.imshow("Gaussian Filtered", gaussian_filtered)
cv.imshow("Median Filtered", median_filtered)
cv.waitKey(0)
cv.destroyAllWindows()

### Average Filtered
- looks blurrier and smooth
- Gaussian noise

### Median Filtered
- shows fewer noise spots and retains the overall structure well
- impulse noise

### Gaussian Filtered
- show some noise reduction
- Gaussian noise 